In [1]:
import pandas as pd

In [5]:
# Load the processed metadataset
metadataset = pd.read_csv('../data/processed/metadata.csv', index_col=0)
metadataset

,cloud,land,missing_landsat,kelp,in_train
tile_id,,,,,
JW725114,0.008294,2.849404,0.000000,0.000082,True
UX493605,0.004155,8.083894,0.000000,0.007404,True
OU500661,0.039673,2.102188,0.000000,0.000000,True
DC227980,0.009371,13.158114,0.000000,0.000000,True
SS602790,0.061763,53.465535,0.000000,0.000000,True
...,...,...,...,...,...
UT495238,0.297796,39.804449,0.000008,NaN,False
GE987629,0.307053,4.148800,0.000008,NaN,False
EN974536,0.348498,43.202147,0.000008,NaN,False


TODO: Load prediction results csv to analyze performance